In [1]:
token = ""

# 1. Load and Transform Dataset

In [2]:
from datasets import load_dataset

DATASET_NAME = "TIGER-Lab/MathInstruct"
dataset = load_dataset(DATASET_NAME, split='train')

/root/miniconda3/envs/unsloth/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
if False:
    from transformers import AutoTokenizer

    model_id = "gg-hf/gemma-2b-it"
    tokenizer = AutoTokenizer.from_pretrained(model_id, token=token)
    model_turn_identifier = '<start_of_turn>model\n'

    lengths = []
    for idx, item in enumerate(ds_valid):
        length = len(tokenizer.tokenize(item['text'].split(model_turn_identifier)[1]))
        lengths.append(length)
        print(idx)
        if idx > 50000:
            break

In [4]:
if False:
    import matplotlib.pyplot as plt

    plt.hist(lengths, range=[0,1500], bins=50, density=True, cumulative=True)
    plt.show()

In [5]:
# Remove duplicates

from collections import Counter

instruction_counts = Counter(dataset['instruction'])
duplicates = {instruction: count for instruction, count in instruction_counts.items() if count > 1}
def filter_unique_examples(example):
    return instruction_counts[example['instruction']] == 1

filtered_dataset = dataset.filter(filter_unique_examples)

In [6]:
len(filtered_dataset), len(dataset)

(208826, 262039)

In [7]:
# Transform data
from transformers import AutoTokenizer

system_prompt = "You're a helpful assistant that answers math problems."
model_id = "google/gemma-2-2b-it"
tokenizer = AutoTokenizer.from_pretrained(model_id, token=token)

def formatting_prompt(item):
    chat = [
        { "role": "user", "content": system_prompt + '\n' + item['instruction'] },
        { "role": "model", "content": item['output'] }
    ]
    prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=False)
    return {'text': prompt}
    
formatted_dataset = filtered_dataset.map(formatting_prompt)

In [8]:
formatted_dataset[0]

{'source': 'data/CoT/aqua_rat.json',
 'output': "Let's think about the multi-choice question.\n6.52 × 10^5 ly / (3.26 ly/parsec) = 2 x 10^5 persec\nThe answer is A.",
 'instruction': 'The distance between two stars is 6.52 × 10^5 light years. What is the distance between the two stars in parsecs? (1 parsec = 3.26 light years)\nAnswer Choices: (A) 2 × 10^5 (B) 4 × 10^6 (C) 5 × 10^7 (D) 7 × 10^7 (E) 9 × 10^8',
 'text': "<bos><start_of_turn>user\nYou're a helpful assistant that answers math problems.\nThe distance between two stars is 6.52 × 10^5 light years. What is the distance between the two stars in parsecs? (1 parsec = 3.26 light years)\nAnswer Choices: (A) 2 × 10^5 (B) 4 × 10^6 (C) 5 × 10^7 (D) 7 × 10^7 (E) 9 × 10^8<end_of_turn>\n<start_of_turn>model\nLet's think about the multi-choice question.\n6.52 × 10^5 ly / (3.26 ly/parsec) = 2 x 10^5 persec\nThe answer is A.<end_of_turn>\n"}

In [9]:
split_dataset = formatted_dataset.train_test_split(test_size=0.005, seed=3407)

ds_train = split_dataset['train']
ds_valid = split_dataset['test']

In [10]:
ds_valid

Dataset({
    features: ['source', 'output', 'instruction', 'text'],
    num_rows: 1045
})

# 2. Model Evaluation

In [11]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 512 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name = "unsloth/gemma-2-2b-it-bnb-4bit",
#     max_seq_length = max_seq_length,
#     dtype = dtype,
#     # load_in_4bit = load_in_4bit,
#     token = token
# )

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [12]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "outputs/checkpoint-10590", # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = max_seq_length,
    # dtype = dtype,
    # load_in_4bit = load_in_4bit,
)

Unsloth: If you want to finetune Gemma 2, install flash-attn to make it faster!
To install flash-attn, do the below:

pip install --no-deps --upgrade "flash-attn>=2.6.3"
==((====))==  Unsloth 2024.8: Fast Gemma2 patching. Transformers = 4.45.0.dev0.
   \\   /|    GPU: NVIDIA GeForce RTX 3080 Ti. Max memory: 11.73 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.2+cu121. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.25.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unsloth 2024.8 patched 26 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [ ]:
import json

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

def generate_output(sample):
    inputs = tokenizer([sample], return_tensors = "pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens = 800, use_cache = True)
    return tokenizer.batch_decode(outputs)[0]

model_turn_identifier = '<start_of_turn>model\n'

with open('./gemma2_finetuned_response.jsonl', 'w+') as f:
    for idx, item in enumerate(ds_valid):
        if idx > 500:
            break
        print(f"{idx} processing out of {len(ds_valid)}")
        inference_prompt = item['text'].split(model_turn_identifier)[0] + model_turn_identifier
        response = generate_output(inference_prompt)
        response = response.split(model_turn_identifier)[1].strip()
        f.write(json.dumps({"index": idx, 
                            "groundtruth": item['text'].split(model_turn_identifier)[1], 
                            "prediction": response}) + '\n')